In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('Linear regression example').getOrCreate()

In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
df = spark.read.csv('dbfs:/FileStore/shared_uploads/abhishekbedarkar28@gmail.com/Ecommerce_Customers.csv', inferSchema=True, header=True)

In [0]:
df.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [0]:
df.show(1)

+--------------------+--------------------+------+------------------+-----------------+-----------------+--------------------+-------------------+
|               Email|             Address|Avatar|Avg Session Length|      Time on App|  Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+------+------------------+-----------------+-----------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|Violet| 34.49726772511229|12.65565114916675|39.57766801952616|  4.0826206329529615|  587.9510539684005|
+--------------------+--------------------+------+------------------+-----------------+-----------------+--------------------+-------------------+
only showing top 1 row



In [0]:
df.columns

Out[11]: ['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
assembler = VectorAssembler(inputCols=['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership'], outputCol='features')
output = assembler.transform(df)

In [0]:
output.select('features').show(1)

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
+--------------------+
only showing top 1 row



In [0]:
model_data = output.select('features','Yearly Amount Spent')
model_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [0]:
train_data, test_data = model_data.randomSplit([0.7,0.3])

In [0]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                341|
|   mean|  494.7407276822555|
| stddev|  77.47639570100674|
|    min| 256.67058229005585|
|    max|  725.5848140556806|
+-------+-------------------+



In [0]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                159|
|   mean|  509.1222074820521|
| stddev|  82.51648499151395|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+



In [0]:
lr = LinearRegression(labelCol='Yearly Amount Spent', featuresCol='features')

In [0]:
lr_model = lr.fit(train_data)

In [0]:
test_results = lr_model.evaluate(test_data)

In [0]:
test_results.r2

Out[40]: 0.9838781731112312

In [0]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-10.537019161045805|
|-16.099392287299793|
|  4.731029618858088|
| 12.019622691950417|
| 3.8737571669743716|
| -4.845397682960993|
| -5.157908306975628|
|  4.789802171994381|
|-16.146503024733875|
| 18.925942468347785|
|-1.2240820487208453|
|  2.606091599307149|
|-1.7930321164772067|
|  -8.87659494522535|
|  11.94383427220356|
|  6.031900844450661|
| 23.141549500151598|
|    5.7998381377638|
| -2.578362074398683|
| -5.392039807839637|
+-------------------+
only showing top 20 rows



In [0]:
unlabled_data = test_data.select('features')

In [0]:
unlabled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.3931845423455...|
|[30.8162006488763...|
|[31.0472221394875...|
|[31.1695067987115...|
|[31.4459724827577...|
|[31.5147378578019...|
|[31.5257524169682...|
|[31.5316044825729...|
|[31.5702008293202...|
|[31.6005122003032...|
|[31.7216523605090...|
|[31.7366356860502...|
|[31.8530748017465...|
|[31.8854062999117...|
|[31.9262720263601...|
|[32.0478009788678...|
|[32.0498393904573...|
|[32.0542618511847...|
|[32.0775900443291...|
|[32.0883806304482...|
+--------------------+
only showing top 20 rows



In [0]:
predictions = lr_model.transform(unlabled_data)

In [0]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...| 330.4658889642394|
|[30.8162006488763...| 282.1857332357688|
|[31.0472221394875...| 387.7663695701633|
|[31.1695067987115...| 415.3369081103424|
|[31.4459724827577...| 481.0032077681542|
|[31.5147378578019...| 494.6578856794224|
|[31.5257524169682...| 449.1235351168575|
|[31.5316044825729...| 431.7258035573682|
|[31.5702008293202...| 562.0919951661388|
|[31.6005122003032...|460.24690902274915|
|[31.7216523605090...| 349.0010086805935|
|[31.7366356860502...|494.32735465622477|
|[31.8530748017465...| 461.0781555788292|
|[31.8854062999117...|398.97986791770086|
|[31.9262720263601...|380.26109917212284|
|[32.0478009788678...| 507.4186703416458|
|[32.0498393904573...| 455.5778073740637|
|[32.0542618511847...| 556.0748195312192|
|[32.0775900443291...| 403.6114972935047|
|[32.0883806304482...| 517.5579061959977|
+--------------------+------------